In [3]:
import numpy as np
import cs_vqe as c
import ast
import os

from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp

In [4]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [5]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [6]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [7]:
# mol_key = 'H2_6-31G_singlet'  
# mol_key ='H2-O1_STO-3G_singlet'
mol_key = 'H1-F1_STO-3G_singlet'


# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'IIIIIIII': -65.32093253713555,
 'IIIIIIIX': 0.002402667690835772,
 'IIIIIIIZ': 16.180098271882343,
 'IIIIIIXI': -0.040871854698728574,
 'IIIIIIXZ': -0.05737873582321127,
 'IIIIIIYY': -0.006360598179958599,
 'IIIIIIZI': 16.180098271882343,
 'IIIIIIZZ': 1.340646573147166,
 'IIIIIXIX': -0.12644597509546893,
 'IIIIIXXI': 0.0039802291595885655,
 'IIIIIXYY': 0.006623525971099688,
 'IIIIIXZX': -0.16768107055914974,
 'IIIIIXZZ': -0.01659011980304869,
 'IIIIIYIY': -0.12644597509546893,
 'IIIIIYYX': -0.006623525971099688,
 'IIIIIYZY': -0.16768107055914974,
 'IIIIIZII': 2.3357257902560375,
 'IIIIIZIZ': 0.28161764927929583,
 'IIIIIZXZ': -0.0001048259582169446,
 'IIIIIZZI': 0.30048077113694405,
 'IIIIXIXI': -0.004988548097378894,
 'IIIIXIZZ': -0.031703692461981056,
 'IIIIXXIX': -0.001323027644611646,
 'IIIIXXYY': -0.01886312185764844,
 'IIIIXYIY': -0.001323027644611646,
 'IIIIXYYX': 0.01886312185764844,
 'IIIIXZIZ': -0.053180628872309704,
 'IIIIXZXI': -0.16768107055914966,
 'IIIIXZXZ': -0.1264459

In [6]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  8


# Get non-contextual H

In [7]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

Split into:

$$H = H_{c} + H_{nc}$$

In [8]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [9]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [10]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [11]:
Z_list

['IIIIIIII',
 'IIIIIIIZ',
 'IIIIIIZI',
 'IIIIIIZZ',
 'IIIIIZII',
 'IIIIIZIZ',
 'IIIIIZZI',
 'IIIIZIII',
 'IIIIZIIZ',
 'IIIIZIZI',
 'IIIIZZII',
 'IIIZIIII',
 'IIIZIIIZ',
 'IIIZIIZI',
 'IIIZIZII',
 'IIIZZIII',
 'IZIIIIII',
 'IZIIIIIZ',
 'IZIIIIZI',
 'IZIIIZII',
 'IZIIZIII',
 'IZIZIIII',
 'IZIZIZIZ',
 'ZIIIIIII',
 'ZIIIIIIZ',
 'ZIIIIIZI',
 'ZIIIIZII',
 'ZIIIZIII',
 'ZIIZIIII',
 'ZIIZIZIZ',
 'ZZIIIIII',
 'ZZIIIZIZ',
 'ZZIIZIZI',
 'ZZIZIIIZ',
 'ZZIZIZII',
 'ZZIZIZIZ',
 'ZZIZIZZZ',
 'ZZIZZZIZ']

In [12]:
T_list

['IIXIIIII',
 'IIXIZZZZ',
 'IIXZIZZZ',
 'IIXZZIZZ',
 'IIXZZZIZ',
 'IIXZZZZI',
 'IIXZZZZZ',
 'IIZIIIII',
 'IIZIIIIZ',
 'IIZIIIZI',
 'IIZIIZII',
 'IIZIZIII',
 'IIZZIIII',
 'IIZZZZZZ',
 'IZXZIZIZ',
 'IZXZZZZZ',
 'IZZIIIII',
 'IZZIZIZI',
 'ZIXZIZIZ',
 'ZIXZZZZZ',
 'ZIZIIIII',
 'ZIZIZIZI',
 'ZZXIIZIZ',
 'ZZXIZIZI',
 'ZZXZIIIZ',
 'ZZXZIZII',
 'ZZXZIZIZ',
 'ZZXZIZZZ',
 'ZZXZZZIZ',
 'ZZZIIIZI',
 'ZZZIZIII',
 'ZZZIZIZI',
 'ZZZIZIZZ',
 'ZZZIZZZI',
 'ZZZZIZIZ',
 'ZZZZZIZI']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [13]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [14]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['IIIIIIII', 'IIIIIIIZ', 'IIIIIIZI', 'IIIIIIZZ', 'IIIIIZII', 'IIIIIZIZ', 'IIIIIZZI', 'IIIIZIII', 'IIIIZIIZ', 'IIIIZIZI', 'IIIIZZII', 'IIIZIIII', 'IIIZIIIZ', 'IIIZIIZI', 'IIIZIZII', 'IIIZZIII', 'IZIIIIII', 'IZIIIIIZ', 'IZIIIIZI', 'IZIIIZII', 'IZIIZIII', 'IZIZIIII', 'IZIZIZIZ', 'ZIIIIIII', 'ZIIIIIIZ', 'ZIIIIIZI', 'ZIIIIZII', 'ZIIIZIII', 'ZIIZIIII', 'ZIIZIZIZ', 'ZZIIIIII', 'ZZIIIZIZ', 'ZZIIZIZI', 'ZZIZIIIZ', 'ZZIZIZII', 'ZZIZIZIZ', 'ZZIZIZZZ', 'ZZIZZZIZ']
G (independent) Z list: ['ZIIIIIII', 'IZIIIIII', 'IIIZIIII', 'IIIIZIII', 'IIIIIZII', 'IIIIIIZI', 'IIIIIIIZ']


In [15]:
print('all Ci1 terms:', Ci1_list)

all Ci1 terms: ['IIZIIIII', 'IIXIIIII']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [16]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'IIIIIIII': [[], [], 1],
 'IIIIIIIZ': [['IIIIIIIZ'], [], 1],
 'IIIIIIZI': [['IIIIIIZI'], [], 1],
 'IIIIIIZZ': [['IIIIIIZI', 'IIIIIIIZ'], [], 1],
 'IIIIIZII': [['IIIIIZII'], [], 1],
 'IIIIIZIZ': [['IIIIIZII', 'IIIIIIIZ'], [], 1],
 'IIIIIZZI': [['IIIIIZII', 'IIIIIIZI'], [], 1],
 'IIIIZIII': [['IIIIZIII'], [], 1],
 'IIIIZIIZ': [['IIIIZIII', 'IIIIIIIZ'], [], 1],
 'IIIIZIZI': [['IIIIZIII', 'IIIIIIZI'], [], 1],
 'IIIIZZII': [['IIIIZIII', 'IIIIIZII'], [], 1],
 'IIIZIIII': [['IIIZIIII'], [], 1],
 'IIIZIIIZ': [['IIIZIIII', 'IIIIIIIZ'], [], 1],
 'IIIZIIZI': [['IIIZIIII', 'IIIIIIZI'], [], 1],
 'IIIZIZII': [['IIIZIIII', 'IIIIIZII'], [], 1],
 'IIIZZIII': [['IIIZIIII', 'IIIIZIII'], [], 1],
 'IIXIIIII': [[], ['IIXIIIII'], 1],
 'IIXIZZZZ': [['IIIIZIII', 'IIIIIZII', 'IIIIIIZI', 'IIIIIIIZ'],
  ['IIXIIIII'],
  1],
 'IIXZIZZZ': [['IIIZIIII', 'IIIIIZII', 'IIIIIIZI', 'IIIIIIIZ'],
  ['IIXIIIII'],
  1],
 'IIXZZIZZ': [['IIIZIIII', 'IIIIZIII', 'IIIIIIZI', 'IIIIIIIZ'],
  ['IIXIIIII'],
  1],
 'IIXZZZIZ': [['IIIZ

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [17]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [18]:
fn_form

[7,
 2,
 [[-65.32093253713555, [], [], 'IIIIIIII'],
  [16.180098271882343, [6], [], 'IIIIIIIZ'],
  [16.180098271882343, [5], [], 'IIIIIIZI'],
  [1.340646573147166, [5, 6], [], 'IIIIIIZZ'],
  [2.3357257902560375, [4], [], 'IIIIIZII'],
  [0.28161764927929583, [4, 6], [], 'IIIIIZIZ'],
  [0.30048077113694405, [4, 5], [], 'IIIIIZZI'],
  [2.3357257902560375, [3], [], 'IIIIZIII'],
  [0.30048077113694405, [3, 6], [], 'IIIIZIIZ'],
  [0.28161764927929583, [3, 5], [], 'IIIIZIZI'],
  [0.21432034653989088, [3, 4], [], 'IIIIZZII'],
  [1.7429834522356595, [2], [], 'IIIZIIII'],
  [0.24332642530783588, [2, 6], [], 'IIIZIIIZ'],
  [0.2490134432861977, [2, 5], [], 'IIIZIIZI'],
  [0.14867454863350857, [2, 4], [], 'IIIZIZII'],
  [0.1877966636992112, [2, 3], [], 'IIIZZIII'],
  [-0.003769886535509768, [], [1], 'IIXIIIII'],
  [0.030527327562793125, [3, 4, 5, 6], [1], 'IIXIZZZZ'],
  [0.030492590951318014, [2, 4, 5, 6], [1], 'IIXZIZZZ'],
  [0.04246247393562265, [2, 3, 5, 6], [1], 'IIXZZIZZ'],
  [0.08792735055124

In [19]:
Energy_function = c.energy_function(fn_form)

In [20]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [21]:
 Energy_function(*q_variables,*r_variables)

-92.68319039493649

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [22]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-103.55745152883416
[[-1, -1, -1, -1, -1, -1, -1], [-0.9999999999999752, 2.223729144271766e-07]]


In [23]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [24]:
ground_state_params

[[-1, -1, -1, -1, -1, -1, -1], [-0.9999999999999752, 2.223729144271766e-07]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [25]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIIIIIII', 'IZIIIIII', 'IIIZIIII', 'IIIIZIII', 'IIIIIZII', 'IIIIIIZI', 'IIIIIIIZ']
['IIZIIIII', 'IIXIIIII']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

# NEW LCU method

In [26]:
N_index=0
check_reduction=True
N_Qubits= hamiltonians[mol_key][1]
R_LCU, Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops= c.diagonalize_epistemic_LCU(
                                                                                  model,
                                                                                  fn_form,
                                                                                  ground_state_params,
                                                                                  N_Qubits,
                                                                                  N_index,
                                                                                  check_reduction=check_reduction)

In [27]:
R_LCU

[0.9238795325112867 [], 0.3826834323650898j [Y2]]

In [28]:
diagonalized_generators_GuA

['ZIIIIIII',
 'IZIIIIII',
 'IIIZIIII',
 'IIIIZIII',
 'IIIIIZII',
 'IIIIIIZI',
 'IIIIIIIZ',
 'IIZIIIII']

In [29]:
diagonalized_generators_GuA

['ZIIIIIII',
 'IZIIIIII',
 'IIIZIIII',
 'IIIIZIII',
 'IIIIIZII',
 'IIIIIIZI',
 'IIIIIIIZ',
 'IIZIIIII']

In [30]:
R_LCU_str = conv_scr.Openfermion_to_dict(R_LCU, N_Qubits)
for op1 in diagonalized_generators_GuA[:-1]:
    for op2 in R_LCU_str:
        print(op1, op2, c.commute(op1, op2))
    print('##')
# should commute with everything BAR script A term (last check) (hence slice ending at [:-1] !!!)

ZIIIIIII IIIIIIII 1
ZIIIIIII IIYIIIII 1
##
IZIIIIII IIIIIIII 1
IZIIIIII IIYIIIII 1
##
IIIZIIII IIIIIIII 1
IIIZIIII IIYIIIII 1
##
IIIIZIII IIIIIIII 1
IIIIZIII IIYIIIII 1
##
IIIIIZII IIIIIIII 1
IIIIIZII IIYIIIII 1
##
IIIIIIZI IIIIIIII 1
IIIIIIZI IIYIIIII 1
##
IIIIIIIZ IIIIIIII 1
IIIIIIIZ IIYIIIII 1
##


In [31]:
eigen_vals_nonC_ground_state_GuA_ops

array([-1, -1, -1, -1, -1, -1, -1,  1])

In [32]:
order = list(range(hamiltonians[mol_key][1])) # [4, 3, 1, 2, 0]#
N_index=0
check_reduction=True
N_Qubits= hamiltonians[mol_key][1]

reduced_H_LCU_list = c.get_reduced_hamiltonians_LCU(ham, # Con_H,
                               model,
                               fn_form,
                               ground_state_params,
                               order, 
                               N_Qubits,
                               N_index, 
                               check_reduction=check_reduction)

In [33]:
reduced_H_LCU_list[-1]

{'IIIIIIII': -65.32093253713555,
 'IIIIIIIX': 0.0024026676908357717,
 'IIIIIIIZ': 16.180098271882343,
 'IIIIIIXI': -0.04087185469872857,
 'IIIIIIXZ': -0.05737873582321127,
 'IIIIIIYY': -0.006360598179958599,
 'IIIIIIZI': 16.180098271882343,
 'IIIIIIZZ': 1.340646573147166,
 'IIIIIXIX': -0.12644597509546893,
 'IIIIIXXI': 0.0039802291595885655,
 'IIIIIXYY': 0.006623525971099688,
 'IIIIIXZX': -0.16768107055914974,
 'IIIIIXZZ': -0.01659011980304869,
 'IIIIIYIY': -0.12644597509546893,
 'IIIIIYYX': -0.006623525971099688,
 'IIIIIYZY': -0.16768107055914974,
 'IIIIIZII': 2.3357257902560375,
 'IIIIIZIZ': 0.2816176492792958,
 'IIIIIZXZ': -0.00010482595821694459,
 'IIIIIZZI': 0.30048077113694405,
 'IIIIXIXI': -0.004988548097378894,
 'IIIIXIZZ': -0.031703692461981056,
 'IIIIXXIX': -0.001323027644611646,
 'IIIIXXYY': -0.01886312185764844,
 'IIIIXYIY': -0.001323027644611646,
 'IIIIXYYX': 0.01886312185764844,
 'IIIIXZIZ': -0.0531806288723097,
 'IIIIXZXI': -0.16768107055914966,
 'IIIIXZXZ': -0.126445975

In [34]:
H = conv_scr.Get_Openfermion_Hamiltonian(reduced_H_LCU_list[-1])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

-103.58635295327144

# Compare to old way!

In [35]:
### old way
order = list(range(hamiltonians[mol_key][1]))
reduced_H_standard_list = c.get_reduced_hamiltonians(ham, # Con_H,
                           model,
                           fn_form,
                           ground_state_params,
                           order)
len(reduced_H_standard_list[0])

1

In [36]:
print(len(reduced_H_LCU_list[-1]), len(reduced_H_standard_list[-1]))

802 802


In [37]:
from Misc_functions import sparse_allclose

H1=conv_scr.Get_Openfermion_Hamiltonian(reduced_H_LCU_list[-1])
H2=conv_scr.Get_Openfermion_Hamiltonian(reduced_H_standard_list[-1])

H1_mat =  qubit_operator_sparse(H1, n_qubits=hamiltonians[mol_key][1])
H2_mat =  qubit_operator_sparse(H2, n_qubits=hamiltonians[mol_key][1])

sparse_allclose(H1_mat, H2_mat)

False

In [38]:
H = conv_scr.Get_Openfermion_Hamiltonian(reduced_H_standard_list[-4])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

-103.56310577850297

In [39]:
from scipy.sparse.linalg import expm
from Misc_functions import sparse_allclose

R_LCU_QubitOp = QubitOperator()
for P in R_LCU: R_LCU_QubitOp+=P
R_LCU_mat = qubit_operator_sparse(R_LCU_QubitOp, n_qubits=hamiltonians[mol_key][1])



R_SeqRot_QubitOp= conv_scr.convert_op_str(Rotations_list[0][1], 1)
R_SeqRot_mat = qubit_operator_sparse(R_SeqRot_QubitOp, n_qubits=hamiltonians[mol_key][1])

theta_sk=Rotations_list[0][0]
exp_rot = expm(R_SeqRot_mat * theta_sk)# / 2)

sparse_allclose(exp_rot, R_LCU_mat)

IndexError: list index out of range

# Restricting the Hamiltonian to a contextualsubspace

(Section B of https://arxiv.org/pdf/2011.10027.pdf)

In the rotated basis the Hamiltonian is restricted to the subspace stabilized by the noncontextual generators $G_{j}'$

In [ ]:
print(diagonalized_generators_GuA) # G_j' terms!

The quantum correction is then obtained by minimizing the expectation value of this resticted Hamiltonian!

(over +1 eigenvectors of the remaining non-contextual generators $\mathcal{A}'$)

In [ ]:
print(Ci1_list) # mathcal(A)

- $\mathcal{H}_{1}$ denotes Hilbert space of $n_{1}$ qubits acted on by by the single qubit $G_{j}'$ terms
- $\mathcal{H}_{2}$ denotes Hilbert space of remaining $n_{2}$

Overall full Hilbert space is: $\mathcal{H}=\mathcal{H}_{1} \otimes \mathcal{H}_{2}$

The **contextual Hamiltonian** in this rotated basis is:

$$H_{c}'=\sum_{P \in \mathcal{S_{c}'}} h_{P}P$$

The set of Pauli terms in $H_{c}'$ is $\mathcal{S_{c}'}$, where terms in $\mathcal{S_{c}'}$ act on both $\mathcal{H}_{1}$ and $\mathcal{H}_{2}$ subspaces in general!

We can write $P$ terms as:

$$P=P_{1}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}}$$

$P$ commutes with an element of $G'$ if and only if $P_{1} \otimes \mathcal{I}^{\mathcal{H}_{2}}$ does

As the generators $G'$ act only on $\mathcal{H}_{1}$

If $P$ anticommutes with any element of $G'$ then its expection value in the noncontextual state is zero

Thus any $P$ must commute with all elements of $G'$ and so $P_{1} \otimes \mathcal{I}^{\mathcal{H}_{2}}$ too

As the elements of $G'$ are single-qubit Pauli $Z$ operators acting in $\mathcal{H}_{1}$:

In [ ]:
print(diagonalized_generators_GuA) # G_j' terms!

$P_{1}$ must be a product of such operators!

**As the exepcation value of $P_{1}$ is some $p_{1}= \pm 1$ DETERMINED BY THE NONCONTEXTUAL GROUND STATE**

In [ ]:
eigen_vals_nonC_ground_state_GuA_ops

Let $|\psi_{(\vec{q}, \vec{r})} \rangle$ be any quantum state consistent with the nonconxtual ground state $(\vec{q}, \vec{r})$... aka gives correct expection values of:

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

Then the action of any $P$ which allows our contextual correction has the form:

$$P |\psi_{(\vec{q}, \vec{r})} \rangle = \big( P_{1}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) |\psi_{(\vec{q}, \vec{r})} \rangle$$

$$ = p_{1}\big( \mathcal{I}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) |\psi_{(\vec{q}, \vec{r})} \rangle$$

- repeating above, but $p_{1}$ is the expectation value of $P_{1}$ determiend by the noncontextual ground state!

Thus we can denote $H_{c}' |_{(\vec{q}, \vec{r})}$ as the restriction of $H_{c}'$ on its action on the noncontextual ground state $(\vec{q}, \vec{r})$:

$$H_{c}' |_{(\vec{q}, \vec{r})} =\sum_{\substack{P \in \mathcal{S_{c}'} \\ \text{s.t.} [P, G_{i}']=0 \\ \forall G'_{i} \in G'}} p_{1}h_{P}\big( \mathcal{I}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) $$

$$=\mathcal{I}_{\mathcal{H}_{1}} \otimes H_{c}'|_{\mathcal{H}_{2}} $$


where we can write:
$$H_{c}'|_{\mathcal{H}_{2}} = \sum_{\substack{P \in \mathcal{S_{c}'} \\ \text{s.t.} [P, G_{i}']=0 \\ \forall G'_{i} \in G'}} p_{1}h_{P}P_{2}^{\mathcal{H}_{2}}$$



Cleary this Hamiltonian on $n_{2}$ qubits is given by:

$$n_{2} = n - |G|$$

- $|G|=$ number of noncontextual generators $G_{j}$

In [ ]:
from copy import deepcopy
import pprint 

```quantum_correction``` function

In [ ]:
n_q = len(diagonalized_generators_GuA[0])

rotated_H = deepcopy(ham) ##<-- full Hamiltonian

# iteratively perform R rotation over all terms in orginal Hamiltonian
for R in Rotations_list:
    newly_rotated_H={}
    for P in rotated_H.keys():
        lin_comb_Rot_P = c.apply_rotation(R,P) # linear combination of Paulis from R rotation on P
        
        for P_rot in lin_comb_Rot_P:
            
            if P_rot in newly_rotated_H.keys():
                newly_rotated_H[P_rot]+=lin_comb_Rot_P[P_rot]*rotated_H[P] # already in it hence +=
            else:
                newly_rotated_H[P_rot]=lin_comb_Rot_P[P_rot]*rotated_H[P]
                
    rotated_H = deepcopy(newly_rotated_H) ##<-- perform next R rotation on this H
    
rotated_H     

next find where Z indices in $G'$

In [ ]:
z_indices = []
for d in diagonalized_generators_GuA:
    for i in range(n_q):
        if d[i] == 'Z':
            z_indices.append(i)
            
print(diagonalized_generators_GuA)
print(z_indices)

**The exepcation value of $P_{1}$ terms are $p_{1}= \pm 1$ DETERMINED BY THE NONCONTEXTUAL GROUND STATE**

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

We need to ENFORCE the diagnal geneators assigned values in the diagonal basis to these expectation values above^^^

In [ ]:
ham_red = {}
    
for P in rotated_H.keys():
    sgn = 1
    for j, z_index in enumerate(z_indices): # enforce diagonal generator's assigned values in diagonal basis
        if P[z_index] == 'Z':
            sgn = sgn*eigen_vals_nonC_ground_state_GuA_ops[j] #<- eigenvalue of nonC ground state!
        elif P[z_index] != 'I':
            sgn = 0

    if sgn != 0:
        # construct term in reduced Hilbert space
        P_red = ''
        for i in range(n_q):
            if not i in z_indices:
                P_red = P_red + P[i]
        if P_red in ham_red.keys():
            ham_red[P_red] = ham_red[P_red] + rotated_H[P]*sgn
        else:
            ham_red[P_red] = rotated_H[P]*sgn
            
ham_red

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
c.quantum_correction(nonCon_H,model,fn_form,ground_state_params)

In [ ]:
c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           list(range(hamiltonians[mol_key][1])))[-1] == rotated_H ### aka when considering all qubit problem it is equal to rotated H!

For some reason it seems that when considering full Hamiltonian there is no reduction in the number of terms!

Q. Do you expect any term reduction when doing CS-VQE?

In [ ]:
n2 = hamiltonians[mol_key][1]-len(diagonalized_generators_GuA)
n2

In [ ]:
ham_red

In [ ]:
ham==Con_H

In [ ]:
n_q = len(diagonalized_generators_GuA[0])
rotated_Hcon = deepcopy(Con_H)

# iteratively perform R rotation over all terms in orginal Hamiltonian
for R in Rotations_list:
    newly_rotated_H={}
    for P in rotated_Hcon.keys():
        lin_comb_Rot_P = c.apply_rotation(R,P) # linear combination of Paulis from R rotation on P
        
        for P_rot in lin_comb_Rot_P:
            
            if P_rot in newly_rotated_H.keys():
                newly_rotated_H[P_rot]+=lin_comb_Rot_P[P_rot]*rotated_Hcon[P] # already in it hence +=
            else:
                newly_rotated_H[P_rot]=lin_comb_Rot_P[P_rot]*rotated_Hcon[P]
                
    rotated_Hcon = deepcopy(newly_rotated_H) ##<-- perform next R rotation on this H
    
rotated_Hcon

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

p1_dict = {Gener.index('Z'): p1 for Gener, p1 in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops)}
p1_dict

In [ ]:
new={}
for P1_P2 in rotated_Hcon.keys():
    Z_indices = [i for i, sigma in enumerate(P1_P2) if sigma=='Z']
    
    I1_P2=list(deepcopy(P1_P2))
    sign=1
    for ind in Z_indices:
        sign*=p1_dict[ind]
        I1_P2[ind]='I'
    I1_P2=''.join(I1_P2)
    
    new[I1_P2]=rotated_Hcon[P1_P2]*sign
    
new    

In [ ]:
len(rotated_Hcon)-len(diagonalized_generators_GuA)
# len(new)

In [ ]:
H = conv_scr.Get_Openfermion_Hamiltonian(new)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
H_reduced_subspace={}
for P in rotated_H.keys():
    
    sign=1
    for P_known, eigen_val in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops):
        Z_index = P_known.index('Z') # Find single qubit Z in generator!
        
        if P[Z_index]== 'Z': # compare location in genertor to P of rotated H
            
            sign*=eigen_val #<- eigenvalue of nonC ground state!
            
        elif P[Z_index]!= 'I': 
            sign=0 # MUST anti-commute!
    
    # build reduced Hilbert Space
    if sign!=0:
        P_new = list(deepcopy(P))
        P_new[Z_index]='I'
        P_new= ''.join(P_new)
        
        if P_new in H_reduced_subspace.keys():
            H_reduced_subspace[P_new] = H_reduced_subspace[P_new] + rotated_H[P]*sign
        else:
            H_reduced_subspace[P_new] = rotated_H[P]*sign
            
#     else:
#         H_reduced_subspace[P]=rotated_H[P]

In [ ]:
print(len(rotated_H))
print(len(H_reduced_subspace))

In [ ]:
# H_reduced_subspace

In [ ]:
lowest_eigenvalue

In [ ]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp
H = conv_scr.Get_Openfermion_Hamiltonian(H_reduced_subspace)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
lowest_eigenvalue

In [ ]:
Hfull = conv_scr.Get_Openfermion_Hamiltonian(ham)
sparseHfull = qubit_operator_sparse(Hfull, n_qubits=hamiltonians[mol_key][1])
FCI = sp.sparse.linalg.eigsh(sparseHfull, which='SA', k=1)[0][0]
print('FCI=', FCI)

In [ ]:
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

p1_dict = {Gener.index('Z'): p1 for Gener, p1 in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops)}
p1_dict

In [ ]:
H_reduced_subspace={}
for P in rotated_Hcon.keys():
    
    new_sign=1
    P_new = list(P)
    for index, sigma in enumerate(P):
        if sigma == 'Z':
            new_sign*=p1_dict[index]
            P_new[index]='I'
    
    P_new = ''.join(P_new)
    H_reduced_subspace[P_new] = rotated_Hcon[P]*new_sign

In [ ]:
H_reduced_subspace

H_con_subspace = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH_con_subspace = qubit_operator_sparse(H_con_subspace, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH_con_subspace, which='SA', k=1)[0][0]

In [ ]:
# H_reduced_subspace={}
# for P in rotated_Hcon.keys():
    
#     p1=1
#     for P_known, eigen_val in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops):
#         Z_index = P_known.index('Z') # Find single qubit Z in generator!
        
#         if P[Z_index]== 'Z': # compare location in genertor to P of rotated H
#             p1*=eigen_val #<- eigenvalue of nonC ground state!
            
#             P1_P2 = list(deepcopy(P))
#             P1_P2[Z_index]='I'
#             I1_P2= ''.join(P1_P2)
            
#             if I1_P2 in H_reduced_subspace.keys():
#                 H_reduced_subspace[I1_P2] += rotated_Hcon[P]*p1
#             else:
#                 H_reduced_subspace[I1_P2] = rotated_Hcon[P]*p1
            
#         elif P[Z_index]== 'I':
#             H_reduced_subspace[P] = rotated_Hcon[P]
            
#         elif P[Z_index]!= 'I': 
#             sign=0 # MUST anti-commute!
#             H_reduced_subspace[P]=0
    
# #         # build reduced Hilbert Space
# #         if sign!=0:
# #             if P_new in H_reduced_subspace.keys():
# #                 H_reduced_subspace[P_new] = H_reduced_subspace[P_new] + rotated_Hcon[P]*sign
# #             else:
# #                 H_reduced_subspace[P_new] = rotated_Hcon[P_new]*sign
            
# # #     else:
# # #         H_reduced_subspace[P]=rotated_H[P]

In [ ]:
H_reduced_subspace

In [ ]:
nonCon_Energy = lowest_eigenvalue

H_con_subspace = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH_con_subspace = qubit_operator_sparse(H_con_subspace, n_qubits=hamiltonians[mol_key][1])
Con_Energy = sp.sparse.linalg.eigsh(sparseH_con_subspace, which='SA', k=1)[0][0]

Con_Energy+nonCon_Energy

In [ ]:
FCI

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
FCI-lowest_eigenvalue

In [ ]:
Con_Energy

In [ ]:
c.commute(P_gen, P)

In [ ]:
rotated_H

In [ ]:
# Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(ham)
Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(Con_H)
rotated_H = QubitOperator()
for P_Rop in R_LCU:
    for P_ham in Ham_openF:
        # left multiply by R
        new_P = P_Rop*P_ham
        rotated_H+=new_P

post_LCU_rot_ham = conv_scr.Openfermion_to_dict(rotated_H,  hamiltonians[mol_key][1])
post_LCU_rot_ham

In [ ]:
len(post_LCU_rot_ham)

In [ ]:
from scipy.linalg import eig
from scipy.linalg import eigh

In [ ]:
Ham_LCU = conv_scr.Get_Openfermion_Hamiltonian(post_LCU_rot_ham)
print(len(list(Ham_LCU)))


sparseH_LCU = qubit_operator_sparse(Ham_LCU, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH_LCU, which='SA', k=1)[0][0]

In [ ]:
eig_values, eig_vectors = eig(sparseH_LCU.todense())
min(eig_values)

In [ ]:
from Misc_functions import sparse_allclose

sparse_allclose(sparseH_LCU, sparseH_LCU.conjugate().T)

In [ ]:
# Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(ham)
newH = {}

HH = Con_H # <--------

for P_ham in HH: # <--------
    # left multiply by R

    rotated_out = c.apply_rotation(Rotations_list[0],P_ham)

    for P_out in rotated_out.keys():
        if P_out in newH.keys():
            newH[P_out] = rotated_out[P_out] + rotated_out[P_out]*HH[P_ham]
        else:
            newH[P_out] = rotated_out[P_out]*HH[P_ham]


In [ ]:
Ham_SEQROT = conv_scr.Get_Openfermion_Hamiltonian(newH)

sparse_SEQROT = qubit_operator_sparse(Ham_SEQROT, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparse_SEQROT, which='SA', k=1)[0][0]

eig_values, eig_vectors = eigh(sparse_SEQROT.todense())
min(eig_values)

In [ ]:
Rotations_list[0]

In [ ]:
from Misc_functions import sparse_allclose

print(sparse_allclose(sparse_SEQROT, sparse_SEQROT.conj().T))
print(sparse_allclose(sparse_SEQROT, sparseH_LCU))

In [ ]:
np.allclose(sparse_SEQROT.todense(), sparseH_LCU.todense())

In [ ]:
from scipy.linalg import eigh
eig_values, eig_vectors = eigh(sparse_SEQROT.todense())
FCI_Energy = min(eig_values)
FCI_Energy

In [ ]:
eig_values, eig_vectors = eigh(sparseH_LCU.todense())
FCI_Energy = min(eig_values)
FCI_Energy

In [ ]:
from scipy.linalg import eig
eig_values, eig_vectors = eig(sparseH_LCU.todense())
FCI_Energy = min(eig_values)
np.abs(FCI_Energy)

In [ ]:
R_LCU_str = conv_scr.Openfermion_to_dict(R_LCU, N_Qubits)
for op1 in Con_H:
    op2 = list(R_LCU_str.keys())[1]
    print(op1, op2, c.commute(op1, op2))
    print('')
# should commute with everything BAR script A term (last check)

In [ ]:
RotOp = QubitOperator()
for R_Rop in R_LCU:
    RotOp+=R_Rop
    
sparseH_RotOp = qubit_operator_sparse(RotOp, n_qubits=hamiltonians[mol_key][1])

from Misc_functions import sparse_allclose
sparse_allclose(sparseH_RotOp, sparseH_RotOp.conjugate().T)

In [ ]:
rot_Hconj= sparseH_RotOp.conjugate().T.todense()
rot_normal = sparseH_RotOp.todense()

np.where(rot_normal!=rot_Hconj)

In [ ]:
i,j = [1,1]
print(rot_Hconj[i,j], rot_normal[i,j])

In [ ]:
rot_Hconj[0,0]
rot_normal[0,0]

In [ ]:
# Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(ham)
Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(Con_H)


manual_rot_H = QubitOperator()
ROP = QubitOperator()
for P_Rop in R_LCU:
    ROP+=P_Rop
    for P_ham in Ham_openF:
        # left multiply by R
        new_P = P_Rop*P_ham
        manual_rot_H+=new_P

openF_rot_H = ROP*Ham_openF

manual_rot_H==openF_rot_H

In [ ]:
Ham_openF*openF_rot_H == openF_rot_H*Ham_openF

In [ ]:
Ham_openF*openF_rot_H == -1*openF_rot_H*Ham_openF

In [ ]:
QubitOperator('Z0')*QubitOperator('X0')

In [ ]:
AC_set = QubitOperator()
for op_str in Ci1_list:
    AC_set+=conv_scr.convert_op_str(op_str, 1)
AC_set.renormalize() #<--- need to renormalize!
AC_set

In [ ]:
from openfermion.utils import hermitian_conjugated
print(ROP)
hermitian_conjugated(ROP)

In [ ]:
ROP*AC_set*hermitian_conjugated(ROP)

In [ ]:
R_mat = qubit_operator_sparse(ROP)

out=R_mat.dot(R_mat.conj().T)
np.allclose(out.todense(), np.eye(2**N_Qubits))

In [ ]:
ROP*hermitian_conjugated(ROP)

In [ ]:
l1_norm = sum([abs(list(op.terms.values())[0]) for op in R_LCU])
ROP_l1_norm = QubitOperator()
for op in ROP:
    PauliStr, const = tuple(*op.terms.items())#
    new_coeff = np.sqrt(const/l1_norm)
    ROP_l1_norm+=QubitOperator(PauliStr, new_coeff)
ROP_l1_norm   

In [ ]:
rot_H_l1_norm = ROP_l1_norm*Ham_openF

l1_H_mat_sparse = qubit_operator_sparse(rot_H_l1_norm)

sparse_allclose(l1_H_mat_sparse, l1_H_mat_sparse.conj().T)

In [ ]:
ROP*hermitian_conjugated(ROP)

In [ ]:
from openfermion.utils import hermitian_conjugated

Ham_openF = conv_scr.Get_Openfermion_Hamiltonian(Con_H)
Ham_openF==hermitian_conjugated(Ham_openF)

In [ ]:
test= ROP*Ham_openF*hermitian_conjugated(ROP)
test==hermitian_conjugated(test)

In [9]:
data_csvqe_results_file = os.path.join(data_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

In [1]:
csvqe_results[mol_key]

NameError: name 'csvqe_results' is not defined

In [42]:
### find optimal LCU qubit removal order!

N_index = 0
check_reduction= True
n_qubits= hamiltonians[mol_key][1]
true_gs= csvqe_results[mol_key][0]

c.csvqe_approximations_heuristic_LCU(ham,
                                   nonCon_H,
                                   n_qubits, 
                                   true_gs, 
                                   N_index, 
                                   check_reduction=check_reduction)

[-103.58635295327042,
 [-102.98191311249451,
  -103.55745152883412,
  -103.57929641696622,
  -103.58146150133092,
  -103.58552552170987,
  -103.58594133415892,
  -103.58633261280683,
  -103.586338665326,
  -103.58635295327052],
 [0.6044398407759104,
  0.02890142443629884,
  0.007056536304204997,
  0.004891451939499802,
  0.0008274315605518723,
  0.00041161911150311425,
  2.034046359256081e-05,
  1.4287944424040688e-05,
  -9.947598300641403e-14],
 [7, 2, 4, 3, 0, 1, 6, 5]]